In [1]:
import tensorrt as trt

In [2]:
import tensorrt as trt

onnx_model_path = "super_resolution.onnx"
engine_file_path = "super_resolution_fp16.engine"

logger = trt.Logger(trt.Logger.WARNING)
builder = trt.Builder(logger)
network = builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH))
parser = trt.OnnxParser(network, logger)

with open(onnx_model_path, "rb") as f:
    if not parser.parse(f.read()):
        for i in range(parser.num_errors):
            print(parser.get_error(i))
        raise RuntimeError("Lỗi parse ONNX")

config = builder.create_builder_config()
config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 1 << 30)

# Tạo optimization profile nếu input dynamic
profile = builder.create_optimization_profile()
input_tensor = network.get_input(0)
input_name = input_tensor.name
profile.set_shape(input_name,
                  min=(1, 3, 128, 128),
                  opt=(1, 3, 256, 256),
                  max=(1, 3, 512, 512))
config.add_optimization_profile(profile)

# Dùng FP16 nếu có thể
if builder.platform_has_fast_fp16:
    config.set_flag(trt.BuilderFlag.FP16)

serialized_engine = builder.build_serialized_network(network, config)
if serialized_engine is None:
    raise RuntimeError("Không thể build TensorRT engine.")

with open(engine_file_path, "wb") as f:
    f.write(serialized_engine)

print("✅ Convert ONNX → TensorRT thành công:", engine_file_path)


✅ Convert ONNX → TensorRT thành công: super_resolution_fp16.engine


In [35]:
import tensorrt as trt
print(trt.__version__)

10.10.0.31


In [2]:
import numpy as np
import cv2
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit  # Tự động khởi tạo CUDA context
import time  # Import thư viện time

# === Load engine TensorRT ===
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
with open("super_resolution_fp16.engine", "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
    engine = runtime.deserialize_cuda_engine(f.read())

context = engine.create_execution_context()

# === Load ảnh gốc (KHÔNG chỉnh sửa kích thước) ===
img = cv2.imread("/home/anansupercuteeeee/Music/sr/super-resolution-lw/dataset/DIV2K_valid_LR_unknown_X4/DIV2K_valid_LR_unknown/X4/0801.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# === Tiền xử lý ảnh ===
input_np = img.astype(np.float32) / 255.0          # Chuẩn hóa về [0, 1]
input_np = np.transpose(input_np, (2, 0, 1))        # HWC -> CHW
input_np = np.expand_dims(input_np, axis=0)         # CHW -> NCHW
input_np = np.ascontiguousarray(input_np)
print(f"Shape của input_np sau tiền xử lý: {input_np.shape}")

# === Tìm index input và output ===
input_idx = -1
output_idx = -1
for i in range(engine.num_io_tensors):
    tensor_name = engine.get_tensor_name(i)
    if engine.get_tensor_mode(tensor_name) == trt.TensorIOMode.INPUT:
        input_idx = i
    elif engine.get_tensor_mode(tensor_name) == trt.TensorIOMode.OUTPUT:
        output_idx = i

if input_idx == -1 or output_idx == -1:
    raise ValueError("Không tìm thấy input hoặc output tensor trong engine.")

# === Lấy tên input ===
input_name = engine.get_tensor_name(input_idx)

# === Set shape đầu vào cho context ===
context.set_input_shape(input_name, input_np.shape)

# === Lấy tên output ===
output_name = engine.get_tensor_name(output_idx)

# === Lấy shape output từ context (sau khi set input shape) ===
output_shape = context.get_tensor_shape(output_name)
print(f"Shape output sau khi set input: {output_shape}")

# === Cấp phát bộ nhớ GPU ===
d_input = cuda.mem_alloc(input_np.nbytes)
output_np = np.empty(output_shape, dtype=np.float32)
d_output = cuda.mem_alloc(output_np.nbytes)

# === Chạy inference và đo thời gian ===
cuda.memcpy_htod(d_input, input_np)
bindings = [0] * engine.num_io_tensors  # Khởi tạo list bindings
bindings[input_idx] = int(d_input)
bindings[output_idx] = int(d_output)

start_time = time.time()  # Ghi lại thời điểm bắt đầu
context.execute_v2(bindings=bindings)
cuda.Context.get_current().synchronize()
end_time = time.time()  # Ghi lại thời điểm kết thúc
inference_time = end_time - start_time
print(f"Thời gian inference: {inference_time:.4f} giây")

cuda.memcpy_dtoh(output_np, d_output)

# === Hậu xử lý ảnh đầu ra ===
output_np = output_np.squeeze()                         # NCHW -> CHW (nếu có batch dimension là 1)
output_np = np.transpose(output_np, (1, 2, 0))          # CHW -> HWC
output_np = np.clip(output_np * 255.0, 0, 255).astype(np.uint8)
output_bgr = cv2.cvtColor(output_np, cv2.COLOR_RGB2BGR)

cv2.imwrite("output_sr.png", output_bgr)
print("✅ Đã lưu ảnh super-res: output_sr.png")

Shape của input_np sau tiền xử lý: (1, 3, 339, 510)
Shape output sau khi set input: (1, 3, 1356, 2040)
Thời gian inference: 0.0144 giây
✅ Đã lưu ảnh super-res: output_sr.png


In [87]:
import cv2
import numpy as np

# Đọc hai ảnh
img1 = cv2.imread('/home/anansupercuteeeee/Music/sr/super-resolution-lw/dataset/DIV2K_valid_HR/0855.png')  # ảnh gốc
img2 = cv2.imread('/home/anansupercuteeeee/Music/sr/super-resolution-lw/output_x4.png')  # ảnh so sánh

# Chuyển về cùng kích thước nếu cần (tuỳ chọn)
if img1.shape != img2.shape:
    img2 = cv2.resize(img2, (img1.shape[1], img1.shape[0]))

# Tính MSE (mean squared error)
mse = np.mean((img1.astype(np.float32) - img2.astype(np.float32)) ** 2)
if mse == 0:
    psnr = float('inf')
else:
    PIXEL_MAX = 255.0
    psnr = 20 * np.log10(PIXEL_MAX / np.sqrt(mse))

print(f"🔍 PSNR giữa hai ảnh: {psnr:.2f} dB")


🔍 PSNR giữa hai ảnh: 23.76 dB


In [103]:
import os
import logging
import cv2
import time
from utils import utils_logger
from utils import utils_image as util

def main():
    utils_logger.logger_info('Bicubic-PSNR-SSIM-X4-strip', log_path='psnr_ssim_bicubic_X4_strip.log')
    logger = logging.getLogger('Bicubic-PSNR-SSIM-X4-strip')

    # ==== Cấu hình thư mục ====
    scale = 4
    lr_folder = 'dataset/DIV2K_valid_LR_unknown_X4/DIV2K_valid_LR_unknown'
    hr_folder = 'dataset/DIV2K_valid_HR'

    lr_paths = util.get_image_paths(lr_folder)
    hr_paths = util.get_image_paths(hr_folder)

    # ==== Tạo từ điển ảnh HR theo tên (không có X4) ====
    hr_dict = {}
    for p in hr_paths:
        name = os.path.splitext(os.path.basename(p))[0]
        hr_dict[name] = p

    psnr_list = []
    ssim_list = []
    runtime_list = []

    for idx, lr_path in enumerate(lr_paths):
        lr_name = os.path.splitext(os.path.basename(lr_path))[0]
        lr_name_clean = lr_name.replace("X4", "").replace("x4", "")  # Bỏ hậu tố X4 hoặc x4
        ext = os.path.splitext(lr_path)[1]

        logger.info("{:>4d} | Đang xử lý ảnh: {}".format(idx + 1, lr_name))

        if lr_name_clean not in hr_dict:
            logger.warning(f"⚠️ Không tìm thấy HR tương ứng cho {lr_name_clean}")
            continue

        # === Load ảnh ===
        img_L = util.imread_uint(lr_path, n_channels=3)
        img_H = util.imread_uint(hr_dict[lr_name_clean], n_channels=3)

        # === Upscale ảnh bằng bicubic ===
        h, w = img_L.shape[:2]
        start = time.time()
        img_E = cv2.resize(img_L, (w * scale, h * scale), interpolation=cv2.INTER_CUBIC)
        end = time.time()

        runtime_list.append(end - start)

        # === Tính PSNR & SSIM ===
        psnr = util.calculate_psnr(img_E, img_H)
        ssim = util.calculate_ssim(img_E, img_H)

        psnr_list.append(psnr)
        ssim_list.append(ssim)

        logger.info(f"✅ {lr_name} | PSNR: {psnr:.2f} dB | SSIM: {ssim:.4f} | Time: {end - start:.4f} s")

    # === Trung bình PSNR, SSIM và thời gian ===
    if psnr_list:
        avg_psnr = sum(psnr_list) / len(psnr_list)
        avg_ssim = sum(ssim_list) / len(ssim_list)
        avg_time = sum(runtime_list) / len(runtime_list)

        logger.info(f"\n📊 Kết quả trung bình trên {len(psnr_list)} ảnh:")
        logger.info(f"   PSNR trung bình (bicubic x{scale}): {avg_psnr:.4f} dB")
        logger.info(f"   SSIM trung bình: {avg_ssim:.4f}")
        logger.info(f"   Thời gian trung bình mỗi ảnh: {avg_time:.6f} giây")
    else:
        logger.warning("❌ Không có ảnh nào được tính PSNR hoặc SSIM.")

if __name__ == '__main__':
    main()


25-05-04 03:17:10.539 :    1 | Đang xử lý ảnh: 0801


LogHandlers setup!


25-05-04 03:17:12.898 : ✅ 0801 | PSNR: 21.46 dB | SSIM: 0.4933 | Time: 0.0079 s
25-05-04 03:17:12.898 :    2 | Đang xử lý ảnh: 0802
25-05-04 03:17:15.150 : ✅ 0802 | PSNR: 26.25 dB | SSIM: 0.7536 | Time: 0.0013 s
25-05-04 03:17:15.158 :    3 | Đang xử lý ảnh: 0803
25-05-04 03:17:17.638 : ✅ 0803 | PSNR: 26.43 dB | SSIM: 0.8597 | Time: 0.0017 s
25-05-04 03:17:17.638 :    4 | Đang xử lý ảnh: 0804
25-05-04 03:17:19.502 : ✅ 0804 | PSNR: 20.12 dB | SSIM: 0.4657 | Time: 0.0011 s
25-05-04 03:17:19.503 :    5 | Đang xử lý ảnh: 0805
25-05-04 03:17:22.042 : ✅ 0805 | PSNR: 22.86 dB | SSIM: 0.5073 | Time: 0.0013 s
25-05-04 03:17:22.043 :    6 | Đang xử lý ảnh: 0806
25-05-04 03:17:25.091 : ✅ 0806 | PSNR: 22.50 dB | SSIM: 0.6903 | Time: 0.0013 s
25-05-04 03:17:25.099 :    7 | Đang xử lý ảnh: 0807
25-05-04 03:17:27.240 : ✅ 0807 | PSNR: 17.38 dB | SSIM: 0.2587 | Time: 0.0015 s
25-05-04 03:17:27.241 :    8 | Đang xử lý ảnh: 0808
25-05-04 03:17:29.448 : ✅ 0808 | PSNR: 21.67 dB | SSIM: 0.6533 | Time: 0.001

In [90]:
import os

def rename_remove_X4(folder):
    for filename in os.listdir(folder):
        name, ext = os.path.splitext(filename)
        if name.endswith("x4"):
            new_name = name[:-2] + ext  # Xóa 'X4' trước đuôi mở rộng
            src = os.path.join(folder, filename)
            dst = os.path.join(folder, new_name)
            os.rename(src, dst)
            print(f"✅ Đã đổi tên: {filename} → {new_name}")

if __name__ == "__main__":
    folder_path = 'dataset/DIV2K_valid_LR_unknown_X4/DIV2K_valid_LR_unknown/X4'
    rename_remove_X4(folder_path)


✅ Đã đổi tên: 0805x4.png → 0805.png
✅ Đã đổi tên: 0809x4.png → 0809.png
✅ Đã đổi tên: 0808x4.png → 0808.png
✅ Đã đổi tên: 0849x4.png → 0849.png
✅ Đã đổi tên: 0877x4.png → 0877.png
✅ Đã đổi tên: 0835x4.png → 0835.png
✅ Đã đổi tên: 0824x4.png → 0824.png
✅ Đã đổi tên: 0867x4.png → 0867.png
✅ Đã đổi tên: 0894x4.png → 0894.png
✅ Đã đổi tên: 0887x4.png → 0887.png
✅ Đã đổi tên: 0845x4.png → 0845.png
✅ Đã đổi tên: 0841x4.png → 0841.png
✅ Đã đổi tên: 0823x4.png → 0823.png
✅ Đã đổi tên: 0855x4.png → 0855.png
✅ Đã đổi tên: 0888x4.png → 0888.png
✅ Đã đổi tên: 0803x4.png → 0803.png
✅ Đã đổi tên: 0876x4.png → 0876.png
✅ Đã đổi tên: 0832x4.png → 0832.png
✅ Đã đổi tên: 0817x4.png → 0817.png
✅ Đã đổi tên: 0822x4.png → 0822.png
✅ Đã đổi tên: 0851x4.png → 0851.png
✅ Đã đổi tên: 0801x4.png → 0801.png
✅ Đã đổi tên: 0868x4.png → 0868.png
✅ Đã đổi tên: 0840x4.png → 0840.png
✅ Đã đổi tên: 0828x4.png → 0828.png
✅ Đã đổi tên: 0881x4.png → 0881.png
✅ Đã đổi tên: 0866x4.png → 0866.png
✅ Đã đổi tên: 0834x4.png → 0

In [86]:
import cv2

# Đọc ảnh gốc
img = cv2.imread("/home/anansupercuteeeee/Music/sr/super-resolution-lw/dataset/DIV2K_valid_LR_unknown_X4/DIV2K_valid_LR_unknown/X4/0855x4.png")  # thay bằng đường dẫn ảnh của bạn

# Lấy kích thước gốc
h, w = img.shape[:2]

# Resize ảnh lên 4 lần (x4)
img_upscaled = cv2.resize(img, (w * 4, h * 4), interpolation=cv2.INTER_CUBIC)

# Lưu ảnh kết quả
cv2.imwrite("output_x4.png", img_upscaled)
print("✅ Đã lưu ảnh sau khi scale x4: output_x4.png")


✅ Đã lưu ảnh sau khi scale x4: output_x4.png


In [102]:
import os
import torch
from utils import utils_image as util
from RFDN import RFDN
import time

def inference_and_eval(input_folder, hr_folder, model_path, device='cuda'):
    model = RFDN()
    model.load_state_dict(torch.load(model_path, map_location=torch.device(device)), strict=True)
    model.eval()
    for p in model.parameters():
        p.requires_grad = False
    model = model.to(device)

    image_list = sorted([f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
    print(f"🔍 Tổng số ảnh cần xử lý: {len(image_list)}")

    total_psnr = 0.0
    total_ssim = 0.0
    count = 0

    for img_name in image_list:
        lr_path = os.path.join(input_folder, img_name)
        hr_path = os.path.join(hr_folder, img_name)

        if not os.path.exists(hr_path):
            print(f"⚠️ Không tìm thấy HR tương ứng: {hr_path}")
            continue

        img_L = util.imread_uint(lr_path, n_channels=3)
        img_H = util.imread_uint(hr_path, n_channels=3)
        img_L_tensor = util.uint2tensor4(img_L).to(device)

        with torch.no_grad():
            start = time.time()
            img_E_tensor = model(img_L_tensor)
            if device == 'cuda':
                torch.cuda.synchronize()
            end = time.time()

        img_E = util.tensor2uint(img_E_tensor)

        psnr = util.calculate_psnr(img_E, img_H)
        ssim = util.calculate_ssim(img_E, img_H)

        total_psnr += psnr
        total_ssim += ssim
        count += 1

        print(f"✅ {img_name} | Time: {end - start:.4f} s | PSNR: {psnr:.2f} dB | SSIM: {ssim:.4f}")

    if count > 0:
        avg_psnr = total_psnr / count
        avg_ssim = total_ssim / count
        print(f"\n📊 Kết quả trung bình trên {count} ảnh:")
        print(f"   PSNR trung bình: {avg_psnr:.2f} dB")
        print(f"   SSIM trung bình: {avg_ssim:.4f}")
    else:
        print("❌ Không có ảnh nào được xử lý hoặc không tìm thấy ảnh HR tương ứng.")

# --- Cấu hình ---
if __name__ == '__main__':
    input_folder = 'dataset/DIV2K_valid_LR_unknown_X4/DIV2K_valid_LR_unknown/X4'
    hr_folder = 'dataset/DIV2K_valid_HR'
    model_path = '/home/anansupercuteeeee/Music/sr/super-resolution-lw/trained_model/RFDN_AIM.pth'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    inference_and_eval(input_folder, hr_folder, model_path, device)


🔍 Tổng số ảnh cần xử lý: 100
✅ 0801.png | Time: 0.0980 s | PSNR: 20.93 dB | SSIM: 0.3985
✅ 0802.png | Time: 0.1017 s | PSNR: 24.51 dB | SSIM: 0.5184
✅ 0803.png | Time: 0.0884 s | PSNR: 24.80 dB | SSIM: 0.6019
✅ 0804.png | Time: 0.0819 s | PSNR: 19.73 dB | SSIM: 0.3681
✅ 0805.png | Time: 0.0884 s | PSNR: 22.20 dB | SSIM: 0.4021
✅ 0806.png | Time: 0.0979 s | PSNR: 21.85 dB | SSIM: 0.4864
✅ 0807.png | Time: 0.0656 s | PSNR: 17.14 dB | SSIM: 0.2011
✅ 0808.png | Time: 0.0979 s | PSNR: 21.15 dB | SSIM: 0.4818
✅ 0809.png | Time: 0.0980 s | PSNR: 23.37 dB | SSIM: 0.4543
✅ 0810.png | Time: 0.0884 s | PSNR: 22.08 dB | SSIM: 0.4197
✅ 0811.png | Time: 0.0980 s | PSNR: 22.10 dB | SSIM: 0.3774
✅ 0812.png | Time: 0.0884 s | PSNR: 20.35 dB | SSIM: 0.4067
✅ 0813.png | Time: 0.1016 s | PSNR: 22.08 dB | SSIM: 0.4816
✅ 0814.png | Time: 0.0979 s | PSNR: 21.85 dB | SSIM: 0.5199
✅ 0815.png | Time: 0.0977 s | PSNR: 23.28 dB | SSIM: 0.5491
✅ 0816.png | Time: 0.0980 s | PSNR: 23.62 dB | SSIM: 0.4799
✅ 0817.png 

In [3]:
import os
import numpy as np
import cv2
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import time

# === Load engine TensorRT ===
TRT_LOGGER = trt.Logger(trt.Logger.WARNING)
with open("super_resolution_fp16.engine", "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
    engine = runtime.deserialize_cuda_engine(f.read())
context = engine.create_execution_context()

# === Tìm index input và output ===
input_idx = -1
output_idx = -1
for i in range(engine.num_io_tensors):
    tensor_name = engine.get_tensor_name(i)
    if engine.get_tensor_mode(tensor_name) == trt.TensorIOMode.INPUT:
        input_idx = i
    elif engine.get_tensor_mode(tensor_name) == trt.TensorIOMode.OUTPUT:
        output_idx = i

if input_idx == -1 or output_idx == -1:
    raise ValueError("Không tìm thấy input hoặc output tensor trong engine.")

input_name = engine.get_tensor_name(input_idx)
output_name = engine.get_tensor_name(output_idx)

# === Đường dẫn folder ảnh đầu vào và đầu ra ===
input_folder = "/home/anansupercuteeeee/Music/sr/super-resolution-lw/dataset/DIV2K_valid_LR_unknown_X4/DIV2K_valid_LR_unknown/X4/"
output_folder = "./results_sr/"
inference_log = "./inference_time_log.txt"
os.makedirs(output_folder, exist_ok=True)

with open(inference_log, 'w') as log_file:
    for filename in sorted(os.listdir(input_folder)):
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue

        # === Load ảnh ===
        img_path = os.path.join(input_folder, filename)
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # === Tiền xử lý ===
        input_np = img.astype(np.float32) / 255.0
        input_np = np.transpose(input_np, (2, 0, 1))
        input_np = np.expand_dims(input_np, axis=0)
        input_np = np.ascontiguousarray(input_np)

        # === Set shape đầu vào ===
        context.set_input_shape(input_name, input_np.shape)

        # === Cấp phát bộ nhớ ===
        d_input = cuda.mem_alloc(input_np.nbytes)
        output_shape = context.get_tensor_shape(output_name)
        output_np = np.empty(output_shape, dtype=np.float32)
        d_output = cuda.mem_alloc(output_np.nbytes)

        # === Binding ===
        bindings = [0] * engine.num_io_tensors
        bindings[input_idx] = int(d_input)
        bindings[output_idx] = int(d_output)

        # === Copy input và chạy inference ===
        cuda.memcpy_htod(d_input, input_np)
        start_time = time.time()
        context.execute_v2(bindings=bindings)
        cuda.Context.get_current().synchronize()
        end_time = time.time()
        inference_time = end_time - start_time
        log_file.write(f"{filename}: {inference_time:.4f} giây\n")
        print(f"{filename} ✅ Inference time: {inference_time:.4f} giây")

        # === Copy output và hậu xử lý ===
        cuda.memcpy_dtoh(output_np, d_output)
        output_np = output_np.squeeze()
        output_np = np.transpose(output_np, (1, 2, 0))
        output_np = np.clip(output_np * 255.0, 0, 255).astype(np.uint8)
        output_bgr = cv2.cvtColor(output_np, cv2.COLOR_RGB2BGR)

        # === Lưu ảnh ===
        out_path = os.path.join(output_folder, filename)
        cv2.imwrite(out_path, output_bgr)

print("🎉 Hoàn tất! Tất cả ảnh đã được xử lý và lưu vào:", output_folder)
print("📄 File thời gian inference:", inference_log)


0801.png ✅ Inference time: 0.0163 giây
0802.png ✅ Inference time: 0.0139 giây
0803.png ✅ Inference time: 0.0147 giây
0804.png ✅ Inference time: 0.0124 giây
0805.png ✅ Inference time: 0.0146 giây
0806.png ✅ Inference time: 0.0140 giây
0807.png ✅ Inference time: 0.0116 giây
0808.png ✅ Inference time: 0.0139 giây
0809.png ✅ Inference time: 0.0139 giây
0810.png ✅ Inference time: 0.0148 giây
0811.png ✅ Inference time: 0.0136 giây
0812.png ✅ Inference time: 0.0148 giây
0813.png ✅ Inference time: 0.0142 giây
0814.png ✅ Inference time: 0.0137 giây
0815.png ✅ Inference time: 0.0137 giây
0816.png ✅ Inference time: 0.0141 giây
0817.png ✅ Inference time: 0.0147 giây
0818.png ✅ Inference time: 0.0147 giây
0819.png ✅ Inference time: 0.0140 giây
0820.png ✅ Inference time: 0.0137 giây
0821.png ✅ Inference time: 0.0140 giây
0822.png ✅ Inference time: 0.0152 giây
0823.png ✅ Inference time: 0.0132 giây
0824.png ✅ Inference time: 0.0138 giây
0825.png ✅ Inference time: 0.0139 giây
0826.png ✅ Inference time